In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sio
import torch.utils.data.dataloader as Data
import os
import time
from torch.autograd import Variable
from DAST_utils import *
from DAST_Network import *
from torch.utils.data import TensorDataset,DataLoader
import sys
from tqdm import tqdm

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
#Myscore function
def myScore(Target, Pred):
    Er = 100 * ((Target - Pred) / Target)
    A = math.exp(-np.log(0.5) * (Er / 5)) if Er <=0 else math.exp(np.log(0.5) * (Er / 20))
    return A

In [4]:
prediction_loss = []
# Load preprocessed data
X_train = sio.loadmat('./Mat_train/Bearing1_1_nor_train_x.mat')  # load sliding window preprocessed and Statistical features processed data (mean value and regression coefficient estimates feature)
X_train = X_train['train_x']

# X_train = X_train.reshape(len(X_train),42,14)
Y_train = sio.loadmat('./Mat_train/Bearing1_1_nor_train_y.mat')
Y_train = Y_train['train_y']
Y_train = Y_train.transpose()

X_test = sio.loadmat('./Mat_test/Bearing1_4_test_x.mat')
X_test = X_test['test_x']
Y_test = sio.loadmat('./Mat_test/Bearing1_4_test_y.mat')
Y_test = Y_test['test_y']
Y_test = Y_test.transpose()

In [5]:
print(X_train.shape)
print(Y_train.shape)

(139800, 40, 3)
(139800, 1)


In [6]:
print(X_test.shape)
print(Y_test.shape)

(1, 40, 3)
(1, 1)


In [6]:
X_train = Variable(torch.Tensor(X_train).float())
Y_train = Variable(torch.Tensor(Y_train).float())
X_test = Variable(torch.Tensor(X_test).float())
Y_test = Variable(torch.Tensor(Y_test).float())

In [11]:
#Hyperparameters
batch_size = 256
dim_val = 32
dim_attn = 32
dim_val_t = 32
dim_attn_t = 32
dim_val_s = 32
dim_attn_s = 32
n_heads = 4
n_decoder_layers = 1
n_encoder_layers = 2
max_rul = 125
lr = 0.001
epochs = 50
time_step = 40
dec_seq_len = 4
output_sequence_length = 1
input_size = 3

In [8]:
#Dataloader
train_dataset = TensorDataset(X_train,Y_train)
train_loader = Data.DataLoader(dataset=train_dataset,batch_size = batch_size,shuffle=True)
test_dataset = TensorDataset(X_test,Y_test)
test_loader = Data.DataLoader(dataset=test_dataset,batch_size = batch_size,shuffle=False)

In [ ]:
# print(train_dataset[0:1])
# print('=' * 60)
# # 循环取数据
# for x_train, y_label in train_dataset:
#     print(x_train, y_label)

In [9]:
# Initialize model parameters
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# debug = True
debug = False
model = DAST(dim_val_s,dim_attn_s,dim_val_t,dim_attn_t,dim_val, dim_attn,time_step,input_size,dec_seq_len,output_sequence_length, n_decoder_layers, n_encoder_layers, n_heads, debug)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.MSELoss()

In [ ]:
# print(next(model.parameters()).device)

In [10]:
#Training  and testing
loss_list = []
train_loss_list = []
test_loss_list = []
train_time = []
test_time = []
model_loss = 1000

In [12]:
for epoch in range(epochs):
    #training
    model.train()
    start1 = time.time()
    loop = tqdm(train_loader, leave=True)
    for i,(X, Y) in enumerate(loop):
        batch_X = X.to(device)
        batch_Y = Y.to(device)
        out = model(batch_X)
        # print(out[0])
        loss = torch.sqrt(criterion(out, batch_Y))
        # print(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    end1 = time.time()
    train_time.append(end1 - start1)
    loss_eopch = np.mean(np.array(loss_list))
    train_loss_list.append(loss_eopch)
    print('epoch = ',epoch,
            'train_loss = ',loss_eopch.item())

    #testing
    model.eval()
    prediction_list = []
    for j ,(batch_x,batch_y) in enumerate(test_loader):
        batch_X = batch_x.to(device)
        batch_Y = batch_y.to(device)
        start2= time.time()
        prediction = model(batch_X)
        end2 = time.time()
        test_time.append(end2 - start2)
        prediction[prediction<0] = 0
        prediction_list.append(prediction)

    out_batch_pre = torch.cat(prediction_list).detach().cpu().numpy()
    prediction_tensor = torch.from_numpy(out_batch_pre)
    test_loss = torch.sqrt(criterion(prediction_tensor, Y_test))
    test_loss_list.append(test_loss.item())
    Y_test_numpy = Y_test.detach().numpy()
    test_score = myScore(Y_test_numpy, out_batch_pre)
    print('test_loss = ', test_loss.item(),
            'test_score = ', test_score)

100%|██████████| 547/547 [00:24<00:00, 21.89it/s]


epoch =  0 train_loss =  1553.897872673746
test_loss =  95.04637145996094 test_score =  1.477491188780546e-17


100%|██████████| 547/547 [00:25<00:00, 21.79it/s]


epoch =  1 train_loss =  1456.9940897437727
test_loss =  281.21270751953125 test_score =  1.599003603267609e-50


100%|██████████| 547/547 [00:25<00:00, 21.47it/s]


...

epoch =  7 train_loss =  705.1971969465015
test_loss =  499.171630859375 test_score =  4.0592786414421005e-89


100%|██████████| 547/547 [00:25<00:00, 21.74it/s]


epoch =  8 train_loss =  642.5501305709737
test_loss =  833.8073120117188 test_score =  2.2507877961488356e-148


 63%|██████▎   | 346/547 [00:14<00:08, 23.52it/s]


KeyboardInterrupt: 

# Appendix

In [ ]:
class test(nn.Module):
    def __init__(self):
        super(test, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
test_model = test()
m = nn.Sequential(nn.Linear(2,2), 
                  nn.ReLU(),
                 nn.Sequential(nn.Sigmoid(), nn.ReLU()))
print(m.children())
print(test_model.children())

In [ ]:
print(type(prediction_list))

In [ ]:
# model.children()
# model._moduless